# < 버트 사전학습 모형 가져오기 >
- [케라스로 버트 학습하기](https://github.com/kimwoonggon/publicservant_AI/blob/master/03_%EC%BC%80%EB%9D%BC%EC%8A%A4%EB%A1%9C_%EB%B2%84%ED%8A%B8_%EB%B9%A0%EB%A5%B4%EA%B2%8C_%EB%8F%8C%EB%A0%A4%EB%B3%B4%EA%B8%B0_With_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_TUTORIAL.ipynb) 참고 사이트
- https://github.com/google-research/bert 에 접속하셔서 ```BERT-Base, Multilingual Cased: 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters``` 파일을 다운

In [50]:
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

from sklearn.model_selection import train_test_split

import codecs
from tqdm import tqdm
import shutil

In [12]:
import os
import warnings
import tensorflow as tf

warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)  # 워닝 뜨는거 감추려고 해주는거 

In [14]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [15]:
os.listdir('bert')

['bert_model.ckpt.data-00000-of-00001',
 '.ipynb_checkpoints',
 'bert_model.ckpt.meta',
 'multi_cased_L-12_H-768_A-12.zip',
 'bert_model.ckpt.index',
 'vocab.txt',
 'bert_config.json']

# df 호출

In [130]:
df = pd.read_csv('data/train.csv', encoding = 'UTF-8-SIG')
# test_df = pd.read_csv('data/test.csv', encoding = 'UTF-8-SIG')

df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


### 확인용

In [115]:
len(df)

8367

In [116]:
print("bias classes: ", df.bias.unique())
print("hate classes: ", df.hate.unique())

bias classes:  ['none' 'others' 'gender']
hate classes:  ['none' 'hate']


In [117]:
# 가능한 모든 조합들의 개수 확인
pd.crosstab(df.bias, df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


# 두 라벨의 가능한 모든 조합 만들기

In [131]:
combinations = np.array(np.meshgrid(df.bias.unique(), df.hate.unique())).T.reshape(-1,2)

combinations

array([['none', 'none'],
       ['none', 'hate'],
       ['others', 'none'],
       ['others', 'hate'],
       ['gender', 'none'],
       ['gender', 'hate']], dtype=object)

## bias, hate 컬럼을 합친 리스트 만들기

In [132]:
bias_hate = list(np.array([df['bias'].values, df['hate'].values]).T.reshape(-1,2))

bias_hate[:5]

[array(['none', 'none'], dtype=object),
 array(['none', 'hate'], dtype=object),
 array(['others', 'hate'], dtype=object),
 array(['none', 'none'], dtype=object),
 array(['none', 'none'], dtype=object)]

## 정수 코드로 라벨 만들기

In [133]:
labels = []

for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

df['label'] = labels
df.head()

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0


# bert 훈련을 위한 사전 설정

In [121]:
SEQ_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-6

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "comment"
LABEL_COLUMN = "label"

분석할 문장이 토큰화가 되고, 그 다음에는 인덱스(숫자)로 변경되어서 버트 신경망에 인풋으로 들어감.

In [122]:
token_dict = {}  # vocab.txt에 있는 단어에 인덱스를 추가해주는 token_dict

with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
            token = token.replace("_","")
            token = "##" + token
        token_dict[token] = len(token_dict)

Tokenizer 클래스를 상속받아서 inherit_Tokennizer라는 클래스를 재정의하고 _tokenize 함수를 새로 작성합니다. 왜 상속을 해서 함수를 새로 만들어야 하냐면, 만약 원래 클래스를 그대로 사용하면 입력한 문장이 "완전자모분리"됩니다. 예를 들어서 "인공지능 할 수 있다" 라는 문장이 있다면 원래 "인,##공,##지,##능,할,수,있다"로 분해가 되어야 하는데, ㅇ,##ㅣ,##ㄴ,ㄱ,##ㅗ,##ㅇ,ㅈ,##ㅣ,##ㄴ,##ㅡ,##ㅇ 이런 식으로 토큰화가 됩니다.

두칸 아래에서 보시겠지만, inherit_Tokenizer클래스는 문장을 토큰화하는 기능을 합니다.

BERT의 토큰화는 단어를 분리하는 토큰화 방식입니다. wordpiece(단어조각?) 방식이라고 하는데, 이는 한국어를 형태소로 꼭 변환해야 할 문제를 해결해주며, 의미가 있는 단어는 밀접하게 연관이 되게 하는 장점까지 갖추고 있습니다.

단어의 첫 시작은 ##가 붙지 않지만, 단어에 포함되면서 단어의 시작이 아닌 부분에는 ##가 붙는 것이 특징입니다.

In [123]:
class inherit_Tokenizer(Tokenizer):
    def _tokenize(self, text):
        if not self._cased:
            text = text
            text = text.lower()
            
        spaced = ''
        
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
                
        tokens = []
        
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
            
        return tokens

In [124]:
tokenizer = inherit_Tokenizer(token_dict)

### 토큰화가 잘 되었는지 확인해 봅니다. 
- 버트 모형은 문장 앞에 꼭 [CLS]라는 문자가 위치하고, [SEP]라는 문자가 끝에 위치합니다.
- [CLS]는 문장의 시작, [SEP]는 문장의 끝을 의미합니다.

In [125]:
tokenizer.tokenize("케라스로 버트 해보기 정말 재밌음.")

['[CLS]',
 '케',
 '##라',
 '##스로',
 '버',
 '##트',
 '해',
 '##보',
 '##기',
 '정',
 '##말',
 '재',
 '##밌',
 '##음',
 '.',
 '[SEP]']

# 감성분석 데이터를 버트 모형의 입력에 맞게 변형해주는 함수를 정의
함수 내부에 tokenizer.encode 함수가 버트 모형을 토큰화해주고 토큰화 된 단어를 인덱스에 맞게 숫자로 바꿔주게 됩니다.

In [147]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    
    return [indices, np.zeros_like(indices)], np.array(targets)


def load_data(data_df):
    # data_df = data_df
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)

    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [155]:
# Define a size for your train set 
train_size = int(0.7 * len(df))

# Split your dataset 
train_df = df[:train_size]
test_df = df[train_size:].reset_index(drop=True)

In [156]:
train_x, train_y = load_data(train_df)
test_x, test_y = load_data(test_df)

100% 5856/5856 [00:00<00:00, 8660.30it/s]
100% 2511/2511 [00:00<00:00, 8885.19it/s]


In [157]:
train_x

[array([[   101,   8935,  83616, ...,      0,      0,      0],
        [   101,   8896, 119449, ...,      0,      0,      0],
        [   101,   9032,  17196, ...,      0,      0,      0],
        ...,
        [   101,    187,  19528, ...,      0,      0,      0],
        [   101,   9764,  53639, ...,      0,      0,      0],
        [   101,   9670,  14523, ...,      0,      0,      0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

- 사전학습된 버트 모델의 인풋은 문장 토큰화가 숫자로 바뀐 것과, 앞문장인지 뒷문장인지 알려주는 문장 순서 벡터가 들어갑니다. 우리는 문장 하나를 가지고만 훈련할 것이므로 순서 벡터는 모두 0으로 통일합니다.

- 그리고 파인튜닝 시에는 문장 안에 일부 단어를 가리는 마스킹은 사용하지 않습니다.
  - Fine Tuning 이란? - 기존에 학습되어져 있는 모델을 기반으로 아키텍쳐를 새로운 목적(나의 이미지 데이터에 맞게)변형하고 이미 학습된 모델 Weights로 부터 학습을 업데이트하는 방법을 말한다.

# bert input = token, segment, position
1. 토큰 : 문장을 토크나이징 한 후, 인덱스 번호를 매긴 것
2. 세그먼트 : 예를 들어 문장이 두 개가 있다면, 앞의 문장과 뒤의 문장을 구분하는 것. 파인튜닝 과정에서 앞뒷문장 구분 안할거면 전부 0으로 지정
3. 포지션 임베딩 : 단순히 단어의 위치를 의미. 단어 순서에 따라서 자동으로 부여됨
-> 토큰, 세그먼트, 포지션을 인풋으로 버트 모형에 넣으면 기하학적인 문장 공간으로 임베딩이 됨

In [171]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    segs = []
    
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        print('segments : ', segments)
        indices.append(ids)
        segs.append(segments)
        
    indices = np.array(indices)
    segs = np.array(segs)
    return [indices, segs]


def sentence_load_data(sentences):  # sentence는 List로 받음
    data_x = sentence_convert_data(sentences)
    return data_x

In [172]:
sentence_load_data(["케라스로 버트 해보기 정말 재밌음. 근데 어렵다", "케라스 쉬워? 쉬워!"])

100% 2/2 [00:00<00:00, 702.33it/s]

	 ['[CLS]', '케', '##라', '##스로', '버', '##트', '해', '##보', '##기', '정', '##말', '재', '##밌', '##음', '.', '근', '##데', '어', '##렵', '##다', '[SEP]']
segments :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 ['[CLS]', '케', '##라', '##스', '쉬', '##워', '?', '쉬', '##워', '!', '[SEP]']
segments :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[array([[   101,   9806,  17342,  94980,   9336,  15184,   9960,  30005,
          12310,   9670,  89523,   9659,    100,  32158,    119,   8926,
          28911,   9546, 118879,  11903,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,   